In [ ]:
#run
# 数据集处理
# imdb
import collections

import numpy as np
import pandas as pd
from openprompt.data_utils.utils import InputExample
import os
import json, csv
from abc import ABC, abstractmethod
from collections import defaultdict, Counter
from typing import List, Dict, Callable

from openprompt.utils.logging import logger
from openprompt.data_utils.data_processor import DataProcessor

class IMDBProcessor(DataProcessor):

    def __init__(self, is_id_only=True, mode=None):
        super().__init__()
        self.labels = None
        self.is_id_only = is_id_only
        self.monitor_mode = 'label'

    def get_examples(self, data_dir: str, split: str, frac: int = 0) -> List[InputExample]:
        examples = []
        if split == 'train' and frac:
            path = os.path.join(data_dir, "{}_{}.csv".format(split, frac))
        else:
            path = os.path.join(data_dir, "{}.csv".format(split))
        data = pd.read_csv(path)
        intents = list(data["intent"])
        # domian_index = list(data['domain_index'])
        indexs = list(data["index"])
        utts = list(data["utt"])
        id_utt = [utts[i] for i in range(len(utts)) if indexs[i] != -1]
        # id_intent = [intents[i] for i in range(len(utts)) if indexs[i] != -1]
        ood_utt = [utts[i] for i in range(len(utts)) if indexs[i] == -1]
        if self.is_id_only:
            utts = id_utt
            is_oods = [0 for _ in range(len(utts))]
            new_indexs = [indexs[i] for i in range(len(indexs)) if indexs[i] != -1]
            # new_domian_index = [domian_index[i] for i in range(len(domian_index)) if domian_index[i] != -1]
        else:
            utts = id_utt + ood_utt
            is_oods = [0 for _ in range(len(id_utt))] + [1 for _ in range(len(ood_utt))]
            new_indexs = [indexs[i] for i in range(len(indexs)) if indexs[i] != -1] + [-1 for _ in range(len(ood_utt))]
            # new_domian_index = [domian_index[i] for i in range(len(domian_index)) if domian_index[i] != -1] + [-1 for _ in range(len(ood_utt))]
            assert len(new_indexs) == len(utts)
            # assert len(new_domian_index) == len(utts)
        if self.monitor_mode == 'label':
            monitor = new_indexs
        # elif self.monitor_mode == 'domain':
        #     monitor = new_domian_index
        for i, (tgt, is_ood, intent) in enumerate(zip(utts, is_oods, new_indexs)):
            example = InputExample(guid=str(i), text_a="", tgt_text=tgt, meta={"is_ood": is_ood},
                                   label=intent if is_ood == 0 else -1)  # label=intent  tgt_text="the intent is"
            examples.append(example)
        return examples

    def get_src_tgt_len_ratio(self, ):
        pass

    def get_label_words(self, data_dir):
        path = os.path.join(data_dir, "train.csv")
        data = pd.read_csv(path)
        intents = list(data["intent"])
        domian_index = list(data['index'])
        result = collections.defaultdict(str)
        for intent, index in zip(intents, domian_index):
            result[index] = intent
        result = sorted(result.items(), key=lambda x: x[0])
        result = [[b] for a, b in result]
        return result

In [ ]:
#run
# clinc
import collections

import numpy as np
import pandas as pd
from openprompt.data_utils.utils import InputExample
import os
import json, csv
from abc import ABC, abstractmethod
from collections import defaultdict, Counter
from typing import List, Dict, Callable

from openprompt.utils.logging import logger
from openprompt.data_utils.data_processor import DataProcessor


class ClincProcessor(DataProcessor):

    def __init__(self, is_id_only=True, mode=None):
        super().__init__()
        self.labels = None
        self.is_id_only = is_id_only
        self.monitor_mode = 'label' if mode == 150 else 'domain'

    def get_examples(self, data_dir: str, split: str, frac: int = 0) -> List[InputExample]:
        examples = []
        if split == 'train' and frac:
            path = os.path.join(data_dir, "{}_{}.csv".format(split, frac))
        else:
            path = os.path.join(data_dir, "{}.csv".format(split))
        data = pd.read_csv(path)
        intents = list(data["intent"])
        domian_index = list(data['domain_index'])
        indexs = list(data["index"])
        utts = list(data["utt"])
        id_utt = [utts[i] for i in range(len(utts)) if indexs[i] != -1]
        # id_intent = [intents[i] for i in range(len(utts)) if indexs[i] != -1]
        ood_utt = [utts[i] for i in range(len(utts)) if indexs[i] == -1]
        if self.is_id_only:
            utts = id_utt
            is_oods = [0 for _ in range(len(utts))]
            new_indexs = [indexs[i] for i in range(len(indexs)) if indexs[i] != -1]
            new_domian_index = [domian_index[i] for i in range(len(domian_index)) if domian_index[i] != -1]
        else:
            utts = id_utt + ood_utt
            is_oods = [0 for _ in range(len(id_utt))] + [1 for _ in range(len(ood_utt))]
            new_indexs = [indexs[i] for i in range(len(indexs)) if indexs[i] != -1] + [-1 for _ in range(len(ood_utt))]
            new_domian_index = [domian_index[i] for i in range(len(domian_index)) if domian_index[i] != -1] + [-1 for _
                                                                                                               in range(
                    len(ood_utt))]
            assert len(new_indexs) == len(utts)
            assert len(new_domian_index) == len(utts)
        if self.monitor_mode == 'label':
            monitor = new_indexs
        elif self.monitor_mode == 'domain':
            monitor = new_domian_index
        for i, (tgt, is_ood, intent) in enumerate(zip(utts, is_oods, monitor)):
            example = InputExample(guid=str(i), text_a="", tgt_text=tgt, meta={"is_ood": is_ood},
                                   label=intent)  # label=intent  tgt_text="the intent is"
            examples.append(example)

        return examples

    def get_src_tgt_len_ratio(self, ):
        pass

    def get_label_words(self, data_dir):
        path = os.path.join(data_dir, "train.csv")
        data = pd.read_csv(path)
        intents = list(data["intent"])
        domian_index = list(data['index'])
        result = collections.defaultdict(str)
        for intent, index in zip(intents, domian_index):
            result[index] = intent
        result = sorted(result.items(), key=lambda x: x[0])
        result = [[b] for a, b in result]
        return result

In [ ]:
# run
import yaml
def load_parameters_from_yaml(file_path):
    with open(file_path, 'r') as file:
        parameters = yaml.safe_load(file)
    return parameters
yaml_file_path = 'paras.yml'
loaded_parameters = load_parameters_from_yaml(yaml_file_path)
config = loaded_parameters

In [ ]:
# run
if config['dataset'] == 'IMDB':
    OOD_DataProcessor = IMDBProcessor
    datasets_dir = "./datasets/imdb_yelp"
    max_seq_length = 256
    batch_size = config['batch_size']

elif config['dataset'] == 'clinc':
    OOD_DataProcessor = ClincProcessor 
    datasets_dir = "./datasets/clinc150/"
    max_seq_length = 128
    batch_size = config['batch_size']


In [ ]:
# run
dataset = {}
dataset['train'] = OOD_DataProcessor(True).get_examples(datasets_dir, "train")
dataset['val'] = OOD_DataProcessor(True).get_examples(datasets_dir, "valid")

dataset["val_ood"] = OOD_DataProcessor(False).get_examples(datasets_dir, "valid")
dataset['test'] = OOD_DataProcessor(False).get_examples(datasets_dir, "test")


In [ ]:
# run
# dataloader
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from typing import List, Dict

class TextClassificationDataset(Dataset):
    def __init__(self, data: List[Dict[str, str]], tokenizer: BertTokenizer, max_length: int = 128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]["tgt_text"]
        label = self.data[idx]["label"]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(data: List[Dict[str, str]], tokenizer: BertTokenizer, batch_size: int = 32, max_length: int = 128):
    dataset = TextClassificationDataset(data, tokenizer, max_length)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return data_loader

def create_test_data_loader(data: List[Dict[str, str]], tokenizer: BertTokenizer, batch_size: int = 32, max_length: int = 128):
    dataset = TextClassificationDataset(data, tokenizer, max_length)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return data_loader

In [ ]:
# run
# 评价指标
import numpy as np
from sklearn import metrics


def compute_all_metrics(conf, label, pred):
    np.set_printoptions(precision=3)
    recall = 0.95
    auroc, aupr_in, aupr_out, fpr = auc_and_fpr_recall(conf, label, recall)

    accuracy = acc(pred, label)

    results = [fpr, auroc, aupr_in, aupr_out, accuracy]

    return results


# accuracy
def acc(pred, label):
    ind_pred = pred[label != -1] #id acc
    ind_label = label[label != -1]

    # ind_pred = pred #all acc
    # ind_label = label

    num_tp = np.sum(ind_pred == ind_label)
    acc = num_tp / len(ind_label)

    return acc


# fpr_recall
def fpr_recall(conf, label, tpr):
    gt = np.ones_like(label)
    gt[label == -1] = 0

    fpr_list, tpr_list, threshold_list = metrics.roc_curve(gt, conf)
    fpr = fpr_list[np.argmax(tpr_list >= tpr)]
    thresh = threshold_list[np.argmax(tpr_list >= tpr)]
    return fpr, thresh


# auc
def auc_and_fpr_recall(conf, label, tpr_th):
    # following convention in ML we treat OOD as positive
    ood_indicator = np.zeros_like(label)
    ood_indicator[label == -1] = 1

    # in the postprocessor we assume ID samples will have larger
    # "conf" values than OOD samples
    # therefore here we need to negate the "conf" values
    
    fpr_list, tpr_list, thresholds = metrics.roc_curve(ood_indicator, -conf)
    fpr = fpr_list[np.argmax(tpr_list >= tpr_th)]

    precision_in, recall_in, thresholds_in \
        = metrics.precision_recall_curve(1 - ood_indicator, conf)

    precision_out, recall_out, thresholds_out \
        = metrics.precision_recall_curve(ood_indicator, -conf)

    auroc = metrics.auc(fpr_list, tpr_list)
    aupr_in = metrics.auc(recall_in, precision_in)
    aupr_out = metrics.auc(recall_out, precision_out)

    return auroc, aupr_in, aupr_out, fpr


# ccr_fpr
def ccr_fpr(conf, fpr, pred, label):
    ind_conf = conf[label != -1]
    ind_pred = pred[label != -1]
    ind_label = label[label != -1]

    ood_conf = conf[label == -1]

    num_ind = len(ind_conf)
    num_ood = len(ood_conf)

    fp_num = int(np.ceil(fpr * num_ood))
    thresh = np.sort(ood_conf)[-fp_num]
    num_tp = np.sum((ind_conf > thresh) * (ind_pred == ind_label))
    ccr = num_tp / num_ind

    return ccr


def detection(ind_confidences,
              ood_confidences,
              n_iter=100000,
              return_data=False):
    # calculate the minimum detection error
    Y1 = ood_confidences
    X1 = ind_confidences

    start = np.min([np.min(X1), np.min(Y1)])
    end = np.max([np.max(X1), np.max(Y1)])
    gap = (end - start) / n_iter

    best_error = 1.0
    best_delta = None
    all_thresholds = []
    all_errors = []
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        detection_error = (tpr + error2) / 2.0

        if return_data:
            all_thresholds.append(delta)
            all_errors.append(detection_error)

        if detection_error < best_error:
            best_error = np.minimum(best_error, detection_error)
            best_delta = delta

    if return_data:
        return best_error, best_delta, all_errors, all_thresholds
    else:
        return best_error, best_delta


In [ ]:
# run
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained('./pretrained_models', padding=True, truncation=True, return_tensors='pt', do_lower_case=True)
model = BertForSequenceClassification.from_pretrained('./pretrained_models', num_labels=config['K'])

train_data = [{"tgt_text": example.tgt_text, "label": example.label} for example in dataset['train']]
val_data = [{"tgt_text": example.tgt_text, "label": example.label} for example in dataset['val']]
test_data = [{"tgt_text": example.tgt_text, "label": example.label} for example in dataset['test']]

train_dataloader = create_data_loader(train_data, tokenizer, batch_size=config['batch_size'], max_length=max_seq_length)
val_dataloader = create_test_data_loader(val_data, tokenizer, batch_size=config['batch_size'], max_length=max_seq_length)
test_dataloader = create_test_data_loader(test_data, tokenizer, batch_size=config['batch_size'], max_length=max_seq_length)


In [ ]:
# run
import torch
import torch.nn as nn
import torch.nn.functional as F

class GRODNet(nn.Module):
    def __init__(self, backbone, feat_dim, num_classes):
        super(GRODNet, self).__init__()

        self.backbone = backbone
        if hasattr(self.backbone, 'fc'):
            # remove fc otherwise ddp will
            # report unused params
            self.backbone.fc = nn.Identity()

        self.lda = LDA(n_components=feat_dim)
        self.pca = PCA(n_components=feat_dim)

        self.n_cls = num_classes
        self.head1 = nn.Linear(768, 2 * num_classes)
        self.head = nn.Linear(768, self.n_cls + 1)
        self.k = nn.Parameter(torch.tensor([0.1], dtype=torch.float32, requires_grad=True))

    def forward(self, x, y, attention): #x:data feature, y:label
        feat = self.backbone.bert(x, attention)[1]#.squeeze() #(b,768)
        # output = self.backbone(x)[0].squeeze() #(b,10)
        self.lda.fit(feat, y)
        X_lda = self.lda.transform(feat) #(b, feat_dim)
        
        self.pca.fit(feat)
        X_pca = self.pca.transform(feat)

        return feat, X_lda, X_pca
    def intermediate_forward(self, x, attention):
        feat = self.backbone.bert(x, attention)[1]
        output = self.head(feat)
        score = torch.softmax(output, dim=1)
        score0 = output[:,:-1]
        # score0 = torch.softmax(output[:,:-1], dim=1)
        conf = torch.max(score, dim=1)
        pred = torch.argmax(score, dim=1)
        conf0 = torch.max(score0, dim=1)
        pred0 = torch.argmax(score0, dim=1)
        for i in range(pred.size(0)):
            if pred[i] == output.size(1) - 1:
                # conf[i] = 0.1
                # pred[i] = 1
                score0[i] = 0.1 * torch.ones(score0.size(1)).cuda() #1/K
            # else:
                # conf[i] = conf0[i]   
        # return score0
        return torch.softmax(score0, dim=1)



class LDA(nn.Module):
    def __init__(self, n_components):
        super(LDA, self).__init__()
        self.n_components = n_components

    def fit(self, X, y):
        try:
            n_samples, n_features = X.shape
        except:
            n_features = X.shape[0]
        classes = torch.unique(y)
        n_classes = len(classes)
        
        means = torch.zeros(n_classes, n_features).to(X.device)
        for i, c in enumerate(classes):
            try:
                means[i] = torch.mean(X[y==c], dim=0)
            except:
                X = torch.unsqueeze(X, dim=0)
                means[i] = torch.mean(X[y==c], dim=0)
        
        overall_mean = torch.mean(X, dim=0)
        
        within_class_scatter = torch.zeros(n_features, n_features).to(X.device)
        for i, c in enumerate(classes):
            class_samples = X[y==c]
            deviation = class_samples - means[i]
            within_class_scatter += torch.mm(deviation.t(), deviation)
        
        between_class_scatter = torch.zeros(n_features, n_features).to(X.device)
        for i, c in enumerate(classes):
            n = len(X[y==c])
            mean_diff = (means[i] - overall_mean).unsqueeze(1)
            between_class_scatter += n * torch.mm(mean_diff, mean_diff.t())

        # torch.backends.cuda.preferred_linalg_library('magma')
        # print((torch.inverse(within_class_scatter) @ between_class_scatter).size()) #(768,768)
        eigenvalues, eigenvectors = torch.linalg.eigh(
        torch.inverse(within_class_scatter @ between_class_scatter  + 1e-2 * torch.eye((within_class_scatter @ between_class_scatter).size(0)).to(X.device)
        ))
        _, top_indices = torch.topk(eigenvalues, k=self.n_components, largest=True)
        self.components = eigenvectors[:, top_indices]

    def transform(self, X):
        return torch.mm(X, self.components)

class PCA(nn.Module):
    def __init__(self, n_components):
        super(PCA, self).__init__()
        self.n_components = n_components

    def fit(self, X):
        try:
            n_samples, n_features = X.shape
        except:
            n_samples = 1
        
        self.mean = torch.mean(X, dim=0)
        X_centered = X - self.mean
        
        covariance_matrix = torch.mm(X_centered.t(), X_centered) / max((n_samples - 1),1)
        
        eigenvalues, eigenvectors = torch.linalg.eigh(covariance_matrix)
        _, top_indices = torch.topk(eigenvalues, k=self.n_components, largest=True)
        self.components = eigenvectors[:, top_indices]

    def transform(self, X):
        X_centered = X - self.mean
        return torch.mm(X_centered, self.components)


In [ ]:
# run
import faiss.contrib.torch_utils
import math
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
from torch.utils.data import DataLoader, Dataset, Subset, TensorDataset
from tqdm import tqdm
from einops import repeat

torch.autograd.set_detect_anomaly(True)
class GRODTrainer:
    def __init__(self, net: nn.Module, train_loader: DataLoader,
                 config) -> None:
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.net = net
        self.train_loader = train_loader
        self.config = config

        self.n_cls = config['dataset']['num_classes']


        self.optimizer = torch.optim.AdamW(
            params=net.parameters(),
            lr=config['optimizer']['lr'],
            weight_decay=config['optimizer']['weight_decay'],
        )

        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max = 10
        )

        self.head = self.net.head
        self.head1 = self.net.head1
        self.alpha = config['trainer']['alpha']
        self.nums_rounded = config['trainer']['nums_rounded']
        self.gamma = config['trainer']['gamma']
        
        self.k = self.net.k
        
        self.best_accuracy = 0.0  # Update best accuracy
        self.best_model_state = None  # Save current model state                
                


    def train(self, epochs):
        # adjust_learning_rate(self.config, self.optimizer, epoch_idx - 1)
        # self.net = nn.DataParallel(self.net)
        self.net.train()
        self.net.to(self.device)
        for epoch_idx in range(epochs):
            loss_avg = 0.0
            train_dataiter = iter(self.train_loader)

            for train_step in tqdm(range(1,
                                        len(train_dataiter) + 1),
                                desc='Epoch {:03d}: '.format(epoch_idx),
                                position=0,
                                leave=True,
                                ):

                batch = next(train_dataiter)
                data = batch['input_ids'].to(self.device)
                target = batch['label'].to(self.device)        
                attention_mask = batch['attention_mask'].to(self.device)   

                if data.size(0) / self.n_cls > 2:
                    sub_datasets_in_mu = torch.zeros((self.n_cls, 768)).to(self.device) #(K,f)
                    
                    dataset_in_mu = torch.zeros(768).to(self.device) #(f)
                    sub_datasets_in_cov = torch.zeros((self.n_cls, 768, 768)).to(self.device)
                    sub_datasets_in_distances = torch.zeros(self.n_cls).to(self.device)
                    data_in, feat_lda, feat_pca = self.net(data, target, attention_mask)
                    
                    # data_in = self.head1(data_in)
                    data = data_in

                    # generate rounded ood data
                    sub_datasets_in = [Subset(data_in, torch.where(target == i)[0]) for i in range(self.n_cls)]
                    sub_datasets_lda = [Subset(feat_lda, torch.where(target == i)[0]) for i in range(self.n_cls)]
                    argmax = torch.zeros((self.n_cls, feat_lda.size()[1]))
                    argmin = torch.zeros((self.n_cls, feat_lda.size()[1]))
                    k = 0
                    for i in range(self.n_cls):
                        if len(sub_datasets_lda[i]) <= 1:
                            pass
                        else:
                            k = k + 1
                            dataloader = DataLoader(sub_datasets_lda[i], batch_size=64, shuffle=False)
                            for batch in dataloader:
                                tensor_data_lda = batch
                            dataloader = DataLoader(sub_datasets_in[i], batch_size=64, shuffle=False)
                            for batch in dataloader:
                                tensor_data_in = batch
                            argmax[i] = torch.argmax(tensor_data_lda,dim=0) #feat_dim
                            argmax[i] = torch.argmin(tensor_data_lda,dim=0) #feat_dim

                            for j in range(feat_lda.size()[1]):
                                # print(argmax[i][j].item())
                                if k == 1 and j==0:
                                    data_rounded_category = tensor_data_in[int(argmax[i][j].item())].unsqueeze(0)
                                    data_rounded_category_1 = tensor_data_in[int(argmin[i][j].item())].unsqueeze(0)
                                else:
                                    data_rounded_category = torch.cat((data_rounded_category, tensor_data_in[int(argmax[i][j].item())].unsqueeze(0)),dim=0)
                                    data_rounded_category_1 = torch.cat((data_rounded_category_1, tensor_data_in[int(argmin[i][j].item())].unsqueeze(0)),dim=0)

                            mean =  torch.mean(tensor_data_in, dim = 0)
                            cov0 = self.calculate_covariance_matrix(tensor_data_in)+1e-4 * torch.eye(mean.size(0)).to(self.device)
                            L = torch.linalg.cholesky(cov0)

                            # 求解下三角矩阵 L 的逆
                            L_inv = torch.linalg.inv(L)

                            # 利用下三角矩阵的逆求解对称正定矩阵 A 的逆
                            cov = torch.mm(L_inv.t(), L_inv)
                            # cov = torch.linalg.inv(self.calculate_covariance_matrix(tensor_data_in)+1e-7 * torch.eye(mean.size(0)).to(self.device))
                            # print(torch.mm(cov0, cov))
                            if torch.max(torch.abs(sub_datasets_in_mu[i,:]))<1e-7:
                                sub_datasets_in_cov[i,:,:] = cov
                                sub_datasets_in_mu[i,:] = mean
                                # print(self.mahalanobis(tensor_data_in, sub_datasets_in_mu, sub_datasets_in_cov))
                                sub_datasets_in_distances[i] = torch.max(self.mahalanobis(tensor_data_in, sub_datasets_in_mu.clone(), sub_datasets_in_cov.clone())[:,i])                                                
                                # print("1",sub_datasets_in_distances[i])
                            
                            sub_datasets_in_cov[i,:,:] = 0.1 * cov.detach().clone().to(self.device) + 0.9 * sub_datasets_in_cov[i,:,:].detach().clone()
                            # print(sub_datasets_in_cov)
                            sub_datasets_in_mu[i,:] = 0.1 * mean.detach().clone().to(self.device) + 0.9 * sub_datasets_in_mu[i,:].detach().clone()
                            dists = self.mahalanobis(tensor_data_in, sub_datasets_in_mu.clone(), sub_datasets_in_cov.clone())[:,i]
                            dist = torch.max(dists)
                            # print(dists.max(), dists.min())
                            sub_datasets_in_distances[i] = 0.1 * dist.to(self.device).detach().clone() + 0.9 * sub_datasets_in_distances[i].detach().clone()
                            # print(i, sub_datasets_in_distances[i])
                            sub_datasets_in_mean =  repeat(sub_datasets_in_mu.clone()[i,:], "f -> b f", 
                                                        f = tensor_data_in.size(1), b = feat_lda.size()[1])
                    
                            A = data_rounded_category[-feat_lda.size()[1]:]
                            A_1 = data_rounded_category_1[- feat_lda.size()[1]:]
                            vector = F.normalize(A.to(self.device) - sub_datasets_in_mean.to(self.device), dim = 1)
                            vector_1 = F.normalize(A_1.to(self.device) - sub_datasets_in_mean.to(self.device), dim = 1)
                            A = A + self.alpha * vector.to(self.device) #(feat_dim, 768)
                            A_1 = A_1 + self.alpha * vector_1.to(self.device) #(feat_dim, 768)
                            if k == 1:
                                mean_matrix_0 = A
                                mean_matrix_1 = A_1
                            else:
                                mean_matrix_0 = torch.cat((mean_matrix_0, A), dim = 0) #(num, 768)
                                mean_matrix_1 = torch.cat((mean_matrix_1, A_1), dim = 0) #(num, 768)
                    mean_matrix = torch.cat((mean_matrix_0, mean_matrix_1), dim = 0)
                    # print(mean_matrix.size())
                    std = 1 / 3 * self.alpha
                    mu = mean_matrix.T.unsqueeze(2).to(self.device) #(768,num,1)
                    
                    # print(mean_matrix.size())
                    
                    rand_data = torch.randn(768, int(self.nums_rounded)).to(self.device) #(768,nums_rounded)
                    gaussian_data = mu + std * rand_data.unsqueeze(1) #(768, num, nums_rounded)
                    # print(gaussian_data.size())
                    nums = gaussian_data.size(1)
                    nums_rounded = gaussian_data.size(2)
                    reshaped_rounded_data = gaussian_data.permute(1, 2, 0).contiguous().view(nums * nums_rounded, mean_matrix.size(1)) # (num* nums_rounded, 768)
                    data = torch.cat((data, reshaped_rounded_data), dim = 0)
                    # print(reshaped_rounded_data.size())

                    # generate PCA ood data
                    argmax = torch.zeros(feat_pca.size()[1])
                    argmax = torch.argmax(feat_pca,dim=0) #feat_dim
                    
                    argmin = torch.zeros(feat_pca.size()[1])
                    argmin = torch.argmin(feat_pca,dim=0) #feat_dim

                    for j in range(feat_pca.size()[1]):
                        if j==0:
                            pcadata_rounded_category = data_in[int(argmax[j].item())].unsqueeze(0)
                            pcadata_rounded_category_1 = data_in[int(argmin[j].item())].unsqueeze(0)
                        else:
                            
                            pcadata_rounded_category = torch.cat((pcadata_rounded_category, data_in[int(argmax[j].item())].unsqueeze(0)),dim=0)
                            pcadata_rounded_category_1 = torch.cat((pcadata_rounded_category_1, data_in[int(argmin[j].item())].unsqueeze(0)),dim=0)
                    if train_step == 1:
                        dataset_in_mu = torch.mean(data_in, dim = 0)
                    dataset_in_mu = 0.1 * torch.mean(data_in.detach().clone(), dim = 0) + 0.9 * dataset_in_mu.detach().clone() 
                    
                    dataset_in_mu =  repeat(dataset_in_mu.squeeze(), "f -> b f", 
                                                    f = data_in.size(1), b = feat_lda.size()[1])
                    # print(data_rounded_category.size())
                    B = pcadata_rounded_category
                    B_1 = pcadata_rounded_category_1
                    # print(A.size())
                    pcavector = F.normalize(B.clone() - dataset_in_mu, dim = 1)
                    pcavector_1 = F.normalize(B_1.clone() - dataset_in_mu, dim = 1)
                    B = torch.add(B, self.alpha * pcavector) #(feat_dim, 768)
                    B_1 = torch.add(B_1, self.alpha * pcavector_1) #(feat_dim, 768)
                    mean_matrix_0 = B
                    mean_matrix_1 = B_1
                    # print(A.size())
                    mean_matrix = torch.cat((mean_matrix_0, mean_matrix_1), dim = 0)
                    # mean_matrix = mean_matrix_0
                    std = 1 / 3 * self.alpha
                    mu = mean_matrix.T.unsqueeze(2).to(self.device) 
                    rand_data = torch.randn(mean_matrix.size(1), self.nums_rounded).to(self.device) 
                    gaussian_data = mu + std * rand_data.unsqueeze(1) #(768, num, nums_rounded)
                    # print(gaussian_data.size())
                    nums = gaussian_data.size(1)
                    nums_rounded = gaussian_data.size(2)
                    reshaped_rounded_data = gaussian_data.permute(1, 2, 0).contiguous().view(nums * nums_rounded, mean_matrix.size(1)) # (num* nums_rounded, 768)
                    # print(reshaped_rounded_data.size(),data.size())
                    data = torch.cat((data, reshaped_rounded_data), dim = 0)
                    # target = torch.cat((target, (self.n_cls) * torch.ones(nums * nums_rounded).to(self.device)), dim = 0)
                    # print(data.size())
                    # filt id like data
                    # print(sub_datasets_in_distances)
                    data_add = data[data_in.size(0):]   
                    # print(data_add.size())
                    
                    distances = self.mahalanobis(data_add, sub_datasets_in_mu, sub_datasets_in_cov).to(self.device) #(n,k)
                    
                    # print(distances, sub_datasets_in_distances)
                    # 计算每个样本点的最小距离和对应的类别索引
                    min_distances, min_distances_clas = torch.min(distances, dim=1)                       
                    # 获取每个样本点对应的子数据集距离
                    sub_distances = sub_datasets_in_distances[min_distances_clas.to(self.device)]
                    
                    k_init = (torch.mean(min_distances / sub_distances) - 1) * 10
                    # 找到满足条件的索引
                    mask = min_distances > (1 + k_init * self.k.to(self.device)[0]) * sub_distances
                    
                    
                    # # 找到满足条件的索引
                    # mask = min_distances > (1 + self.k.to(self.device)[0]) * sub_distances
                    # print(min_distances,sub_distances)
                    # 使用布尔索引删除满足条件的数据点
                    cleaned_data_add = data_add[mask.to(self.device)]
                
                    if cleaned_data_add.size(0) > data_in.size(0) // self.n_cls + 2: #ood太多则随机删除
                        delete_num = cleaned_data_add.size(0) - (data_in.size(0) // self.n_cls + 2)
                        indices = torch.randperm(cleaned_data_add.size(0))[:(data_in.size(0) // self.n_cls + 2)].to(self.device)
                        cleaned_data_add_de = cleaned_data_add[indices]
                    else: 
                        cleaned_data_add_de = cleaned_data_add
                        
                    
                    data = torch.cat((data[:data_in.size(0)], cleaned_data_add_de), dim = 0)


                    target = torch.cat((target, (self.n_cls) * torch.ones(cleaned_data_add_de.size(0)).to(self.device)), dim = 0)
                    
                else:
                    sub_datasets_in_mu = torch.zeros((self.n_cls, 768)).to(self.device) #(K,f)
                    dataset_in_mu = torch.zeros(768).to(self.device) #(f)
                    sub_datasets_in_cov = torch.zeros((self.n_cls, 768, 768)).to(self.device)
                    sub_datasets_in_distances = torch.zeros(self.n_cls).to(self.device)
                    data_in, feat_lda, feat_pca = self.net(data, target, attention_mask)
                    # data_in = self.head1(data_in)
                    data = data_in
                    # generate PCA ood data
                    argmax = torch.zeros(feat_pca.size()[1])
                    argmax = torch.argmax(feat_pca,dim=0) #feat_dim
                    
                    argmin = torch.zeros(feat_pca.size()[1])
                    argmin = torch.argmin(feat_pca,dim=0) #feat_dim

                    for j in range(feat_pca.size()[1]):
                        if j==0:
                            pcadata_rounded_category = data_in[int(argmax[j].item())].unsqueeze(0)
                            pcadata_rounded_category_1 = data_in[int(argmin[j].item())].unsqueeze(0)
                        else:
                            
                            pcadata_rounded_category = torch.cat((pcadata_rounded_category, data_in[int(argmax[j].item())].unsqueeze(0)),dim=0)
                            pcadata_rounded_category_1 = torch.cat((pcadata_rounded_category_1, data_in[int(argmin[j].item())].unsqueeze(0)),dim=0)
                    if train_step == 1:
                            dataset_in_mu = torch.mean(data_in, dim = 0)
                    dataset_in_mu = 0.1 * torch.mean(data_in.detach().clone(), dim = 0) + 0.9 * dataset_in_mu.detach().clone() 
                    cov0 = self.calculate_covariance_matrix(data_in)+1e-4 * torch.eye(dataset_in_mu.size(0)).to(self.device)
                    L = torch.linalg.cholesky(cov0)

                    # 求解下三角矩阵 L 的逆
                    L_inv = torch.linalg.inv(L)

                    # 利用下三角矩阵的逆求解对称正定矩阵 A 的逆
                    cov = torch.mm(L_inv.t(), L_inv)
                    cov = torch.unsqueeze(cov, dim=0)
                    dataset_in_mean =  repeat(dataset_in_mu.squeeze(), "f -> b f", 
                                                    f = data_in.size(1), b = feat_lda.size()[1])
                    # dataset_in_mu = torch.unsqueeze(dataset_in_mu, dim=0)
                    # print(data_rounded_category.size())
                    B = pcadata_rounded_category
                    B_1 = pcadata_rounded_category_1
                    # print(A.size())
                    pcavector = F.normalize(B.clone() - dataset_in_mean, dim = 1)
                    pcavector_1 = F.normalize(B_1.clone() - dataset_in_mean, dim = 1)
                    B = torch.add(B, self.alpha * pcavector) #(feat_dim, 768)
                    B_1 = torch.add(B_1, self.alpha * pcavector_1) #(feat_dim, 768)
                    mean_matrix_0 = B
                    mean_matrix_1 = B_1
                    # print(A.size())
                    mean_matrix = torch.cat((mean_matrix_0, mean_matrix_1), dim = 0)
                    # mean_matrix = mean_matrix_0
                    std = 1 / 3 * self.alpha
                    mu = mean_matrix.T.unsqueeze(2).to(self.device) 
                    rand_data = torch.randn(mean_matrix.size(1), self.nums_rounded).to(self.device) 
                    gaussian_data = mu + std * rand_data.unsqueeze(1) #(768, num, nums_rounded)
                    # print(gaussian_data.size())
                    nums = gaussian_data.size(1)
                    nums_rounded = gaussian_data.size(2)
                    reshaped_rounded_data = gaussian_data.permute(1, 2, 0).contiguous().view(nums * nums_rounded, mean_matrix.size(1)) # (num* nums_rounded, 768)
                    # print(reshaped_rounded_data.size(),data.size())
                    data = torch.cat((data, reshaped_rounded_data), dim = 0)
                    # target = torch.cat((target, (self.n_cls) * torch.ones(nums * nums_rounded).to(self.device)), dim = 0)
                    # print(data.size())
                    # filt id like data
                    # print(sub_datasets_in_distances)
                    data_add = data[data_in.size(0):]   
                    # print(data_add.size())
                
                    distances_add = self.mahalanobis(data_add, dataset_in_mu, cov).to(self.device).squeeze() #(n,1)
                    distance = torch.max(self.mahalanobis(data[:data_in.size(0)], dataset_in_mu, cov).to(self.device))
                    k_init = (torch.mean(distances_add) / distance - 1) * 10
                    # 找到满足条件的索引
                    mask = distances_add > (1 + k_init * self.k.to(self.device)[0]) * distance
                    # print(distances_add, distance)
                    
                    # cleaned_data_add = data_add[mask.to(self.device)]
                    cleaned_data_add = data_add
                    
                    if cleaned_data_add.size(0) > data_in.size(0) // self.n_cls + 2: #ood太多则随机删除
                        delete_num = cleaned_data_add.size(0) - (data_in.size(0) // self.n_cls + 2)
                        indices = torch.randperm(cleaned_data_add.size(0))[:(data_in.size(0) // self.n_cls + 2)].to(self.device)
                        cleaned_data_add_de = cleaned_data_add[indices]
                    else: 
                        cleaned_data_add_de = cleaned_data_add
                    
                
                    data = torch.cat((data[:data_in.size(0)], cleaned_data_add_de), dim = 0)
                    # data = data[:data_in.size(0)]


                    target = torch.cat((target, (self.n_cls) * torch.ones(cleaned_data_add_de.size(0)).to(self.device)), dim = 0)
                    # target = target
                    

                output = self.head(data)
                output = F.normalize(output, dim=1)
                loss1 = F.cross_entropy(output, target.to(torch.long))

                label_matrix = output
                biclas = torch.zeros(label_matrix.size(0), 2)
                biclas[:,-1] = label_matrix[:,-1]
                biclas[:,0] = torch.sum(label_matrix[:,:-1],-1)
                label_biclas = torch.where(
                    torch.gt(target, self.n_cls-0.5),                
                    torch.ones(data.size()[0]).to(self.device),
                    torch.zeros(data.size()[0]).to(self.device),
                    )
                loss2 = F.cross_entropy(biclas.to(self.device), label_biclas.to(torch.int64).to(self.device))

                loss = (1 - self.gamma) * loss1 + self.gamma * loss2
                
                # backward
                self.optimizer.zero_grad()
                # print(loss1,loss2)
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()

                # exponential moving average, show smooth values
                with torch.no_grad():
                    loss_avg = loss_avg * 0.8 + float(loss) * 0.2
            print(f'Epoch {epoch_idx + 1}/{epochs}, Average Training Loss: {loss_avg:.4f}')
            # accuracy = correct / total
            # print(f'Accuracy on validation set: {accuracy:.4f}')
            accuracy = self.test_model()  # Test model after each epoch
            if accuracy > self.best_accuracy or accuracy == self.best_accuracy:  # If current accuracy is better than best
                self.best_accuracy = accuracy  # Update best accuracy
                self.best_model_state = self.net.state_dict()  # Save current model state                
                
    def test_model(self):
        self.net.eval()  # Switch to evaluation mode
        self.net.to(self.device)
        correct = 0
        total = 0
        val_dataiter = iter(self.train_loader)
        with torch.no_grad():
            for train_step in tqdm(range(1, len(val_dataiter) + 1),
                                    position=0,
                                    leave=True):
                batch = next(val_dataiter)
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch['label'].to(self.device)
                
                data_in, feat_lda, feat_pca = self.net(input_ids, labels, attention_mask)
                outputs = self.head(data_in)
                # outputs = self.net.backbone(input_ids, attention_mask).logits
                predicted = torch.argmax(outputs[:,:-1], dim=1)
                # print(predicted, labels)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        print(f'Accuracy on validation set: {accuracy:.4f}')
        return accuracy    

    def save_best_model(self, filename):
        if self.best_model_state is not None:
            torch.save(self.best_model_state, filename)

    
    def mahalanobis(self, x, support_mean, inv_covmat): #(n,d), (k,d), (k,d,d)
        # 获取输入的张量维度信息
        n = x.size(0)
        d = x.size(1)

        # 将输入张量和支持向量均值移到 GPU 上
        x = x.cuda()
        support_mean = support_mean.cuda()

        maha_dists = []
        for i in range(inv_covmat.size(0)):
            class_inv_cov = inv_covmat[i].detach()
            support_class = support_mean[i].detach()
        
            x_mu = x - support_class.unsqueeze(0).expand(n, d)            
            # 将类别协方差矩阵移到 GPU 上
            class_inv_cov = class_inv_cov.cuda()
            # print(torch.max(torch.abs(class_inv_cov)))

            # 计算 Mahalanobis 距离
            left = torch.matmul(x_mu, class_inv_cov)
            # print(left.size())
            mahal = torch.matmul(left, x_mu.t()).diagonal()
            # print('2', mahal) 
            # print(torch.matmul(left, x_mu.t()))
            maha_dists.append(mahal)

        # 将结果转换为 PyTorch 张量
        return torch.stack(maha_dists).t()
    
    def calculate_covariance_matrix(self, data):
        # 计算数据的均值
        mean = torch.mean(data, dim=0)
        mean = mean.unsqueeze(0).expand(data.size(0), data.size(1))
        # 将数据减去均值，得到去中心化的数据
        centered_data = data - mean

        # 计算协方差矩阵
        covariance_matrix = torch.mm(centered_data.t(), centered_data) / (centered_data.size(0) - 1 + 1e-7)
        # print(covariance_matrix.size())
        return covariance_matrix


In [ ]:
yaml_file_path = 'grod.yml'
loaded_parameters = load_parameters_from_yaml(yaml_file_path)
config_grod = loaded_parameters
        
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_grod = GRODNet(model, 1, 2) #(model, num, K)
trainer = GRODTrainer(model_grod, train_dataloader, config_grod)
trainer.train(config_grod['optimizer']['num_epochs']) 

# Save the best model state
trainer.save_best_model('best_model_grod.ckpt') 

In [ ]:
# 加载保存的状态字典
state_dict = torch.load('best_model_grod.ckpt')
model_grod = GRODNet(model, 1, 2)
# 将加载的状态字典加载到模型中
model_grod.load_state_dict(state_dict)
print(model_grod)

In [ ]:
from typing import Any

import numpy as np
import torch
import torch.nn as nn
from numpy.linalg import norm, pinv
from scipy.special import logsumexp
from sklearn.covariance import EmpiricalCovariance
from tqdm import tqdm

# run
from typing import Any
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader


class BasePostprocessor:
    def __init__(self, config):
        self.config = config

    def setup(self, net: nn.Module, id_loader_dict, ood_loader_dict):
        pass

    @torch.no_grad()
    def postprocess(self, net: nn.Module, data: Any, attention):
        output = net(input_ids=data, attention_mask=attention)
        score = torch.softmax(output.logits, dim=1)
        conf, pred = torch.max(score, dim=1)
        return pred, conf

    def inference(self,
                  net: nn.Module,
                  data_loader: DataLoader, 
                  alpha, w, b, u, NS,
                  progress: bool = True):
        pred_list, conf_list, label_list = [], [], []
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].cuda()
            attention_mask = batch["attention_mask"].cuda()
            labels = batch['label'].cuda()
            pred, conf = self.postprocess(net.cuda(), input_ids, attention_mask, alpha, w, b, u, NS,)

            pred_list.append(pred.cpu())
            conf_list.append(conf.cpu())
            label_list.append(labels.cpu())

        # convert values into numpy array
        pred_list = torch.cat(pred_list).numpy().astype(int)
        conf_list = torch.cat(conf_list).numpy()
        label_list = torch.cat(label_list).numpy().astype(int)

        return pred_list, conf_list, label_list


class GRODPostprocessor(BasePostprocessor):
    def __init__(self, config):
        super().__init__(config)
        self.args = self.config['postprocessor']['postprocessor_args']
        self.args_dict = self.config['postprocessor']['postprocessor_sweep']
        self.dim = self.args['dim']
        self.setup_flag = False

    def setup(self, net: nn.Module, id_loader_dict):
        if not self.setup_flag:
            net.eval()
            net.cuda()
            with torch.no_grad():
                # self.w, self.b = net.backbone.get_fc()
                # print(self.b.size)
                self.w, self.b = net.head.weight[:-1,:].cpu().numpy(), net.head.bias[:-1].cpu().numpy()
                # print(self.w.size())
                print('Extracting id training feature')
                feature_id_train = []
                logit_id_train = []
                for batch in tqdm(id_loader_dict,
                                  desc='Setup: ',
                                  position=0,
                                  leave=True):
                    data = batch['input_ids'].cuda()
                    attention_mask = batch["attention_mask"].cuda()
                    labels = batch['label'].cuda()
                    # data = data.float()
                    feature = net.backbone.bert(data, attention_mask)[1]
                    logit = net.head(feature)
                    score = torch.softmax(logit, dim=1)
                    score0 = torch.softmax(logit[:,:-1], dim=1)
                    conf, pred = torch.max(score, dim=1)
                    conf0, pred0 = torch.max(score0, dim=1)
                    for i in range(pred.size(0)):
                        if pred[i] == logit.size(1) - 1:
                            conf[i] = 0.1
                            pred[i] = 1
                            score0[i, :] = 0.1 * torch.ones(score0.size(1)).cuda()
                        else:
                            conf[i] = conf0[i]     
                        
                    feature_id_train.append(feature.cpu().numpy())
                    logit_id_train.append(score0.cpu().numpy())
                feature_id_train = np.concatenate(feature_id_train, axis=0)
                logit_id_train = np.concatenate(logit_id_train, axis=0)

                # logit_id_train = feature_id_train @ self.w.T + self.b

            self.u = -np.matmul(pinv(self.w), self.b)
            ec = EmpiricalCovariance(assume_centered=True)
            ec.fit(feature_id_train - self.u)
            eig_vals, eigen_vectors = np.linalg.eig(ec.covariance_)
            self.NS = np.ascontiguousarray(
                (eigen_vectors.T[np.argsort(eig_vals * -1)[self.dim:]]).T)

            vlogit_id_train = norm(np.matmul(feature_id_train - self.u,
                                             self.NS),
                                   axis=-1)
            
            print(feature_id_train - self.u, self.NS)
            
            self.alpha = logit_id_train.max(
                axis=-1).mean() / vlogit_id_train.mean()
            print(f'{self.alpha=:.4f}')

            self.setup_flag = True
        else:
            pass
        return self.alpha, self.w, self.b, self.u, self.NS
    
    @torch.no_grad()
    def postprocess(self, net: nn.Module, data: Any, attention, alpha, w, b, u, NS):
        feature_ood = net.backbone.bert(data, attention)[1]
        
        logit = net.head(feature_ood)
        score = torch.softmax(logit, dim=1)
        score0 = torch.softmax(logit[:,:-1], dim=1)
        conf, pred = torch.max(score, dim=1)
        conf0, pred0 = torch.max(score0, dim=1)
        for i in range(pred.size(0)):
          if pred[i] == logit.size(1) - 1:
            conf[i] = 0.1
            pred[i] = 1
            score0[i, :] = 0.1 * torch.ones(score0.size(1)).cuda()
          else:
            conf[i] = conf0[i]
        logit_ood = score0.cpu()    
        
        feature_ood = feature_ood.cpu()
        
        # logit_ood = feature_ood @ w.T + b
        _, pred = torch.max(logit_ood, dim=1)
        energy_ood = logsumexp(logit_ood.numpy(), axis=-1)
        vlogit_ood = norm(np.matmul(feature_ood.numpy() - u, NS),
                          axis=-1) * alpha
        score_ood = -vlogit_ood + energy_ood
        return pred, torch.from_numpy(score_ood)

    def set_hyperparam(self, hyperparam: list):
        self.dim = hyperparam[0]

    def get_hyperparam(self):
        return self.dim


In [ ]:
yaml_file_path = 'grod.yml'
loaded_parameters = load_parameters_from_yaml(yaml_file_path)
config_grod = loaded_parameters
alpha, w, b, u, NS = GRODPostprocessor(config_grod).setup(model_grod, train_dataloader)

pred_list, conf_list, label_list = GRODPostprocessor(config_grod).inference(model_grod, test_dataloader, alpha, w, b, u, NS)

# [fpr, auroc, aupr_in, aupr_out, accuracy]
compute_all_metrics(conf_list, label_list, pred_list)